In [9]:
import numpy as np 
import matplotlib.pyplot as plt
import random
import csv

from ipynb.fs.full.coinpress1 import UVMRec

In [10]:
r = 500 #bootstrap iterations

In [11]:
def blb(estimator_func, data, n, true_var, bounds_theta, bounds_sigma_sq):
    
    k = int(4 * np.sqrt(n)) # num partitions
    r = 500 #bootstrap iterations

    # tau: estimators
    tau = np.zeros([k,r]) 

    theta_vec = []
    sigma_sq_vec = []
    # randomly partition X into k subsets
    random.shuffle(data)
    partition_size = int(np.floor(n/k))
    partitions = [data[i:i + partition_size] for i in range(0, len(data), partition_size)]

    # range for randint
    low=0
    high=len(partitions[0]) #don't need to do b-1 because upper bound is exclusive

    # for each partition
    for i in range(k):
    
        b = len(partitions[i]) 
    
        # for a partition, create r subsets
        for c in range(r):
            I = np.random.randint(low, high, size=n)
#             print("I ", I)
            replicate = data[I]
        
            tau[i,c] = estimator_func(replicate)  
           
        low = low + b
        high = high + b
        
    # mean and var of estimators for each prtition, not the data        
    theta_vec = np.mean(tau,axis=1)
    sigma_sq_vec = np.var(tau,axis=1) #var

    #inputs for UVMRec
    t = 10
    rho = 0.5
    beta = 0.1
    
    est_sd_theta = np.sqrt(true_var * 1/len(data))
    
    theta = UVMRec(theta_vec, bounds_theta, est_sd_theta, t, rho, beta)
    
    est_sd_sigma_sq = np.sqrt((2*(np.std(data)**4))/(len(data)-1))
    sigma_sq = UVMRec(sigma_sq_vec, bounds_sigma_sq, est_sd_sigma_sq, t, rho, beta)

    return (theta, sigma_sq)